In [34]:
from utils import profiler, reader
from typing import List, Dict, Tuple, Set
from tqdm import tqdm
from itertools import combinations
from collections import defaultdict


['a', 'b', 'c', 'd']

In [6]:
datafile = "../data/day8_input.txt"
data = reader.read_from_file(datafile)
data = [x.rstrip() for x in data]
data

['...............3................d.................',
 '.........................s..7......i.....e........',
 '................C.......................e.........',
 '...............Z.......m....................e.....',
 '....................gC.....q......................',
 '...............Q....s..........................A..',
 '................................s........A........',
 '...........n.....3.C..F......w..m...d.............',
 '..E...............3.....m......d.i................',
 '............f.3.......C....d........A.............',
 '.........Z...........................n..A.........',
 '....Q......p..............g.i.....................',
 '.r......n...Q....p............S.7...........O.....',
 '..........r......K....p.....M..........7....G.....',
 '....................Fs...................G........',
 '..z.........D..........G.g........................',
 'rR.............F................M...............G.',
 '.........I..c.nr...............M................O.',
 '...I....

# Part 1

### Overview

We need to find all locations of "antinodes" and count them. Any pair of matching characters creates an antinode on opposite sides of each other with vector displacement equal to the displacement between the two matching characters as long as the position is within the grid.

### Approach

We will store those in a hashtable. We find all combinations of pairs of points without replacement. For each one, we'll compute the pairs and their vector displacement. Then we will find the potential antinode position by vector addition componentwise. If that position is in the grid we'll add another antinode.

For instance. Suppose we have {'X': [(x1, y1), (x2, y2), (x3, y3)]}
Then we know the displacement d12 = [x2 - x1, y2 - y1] = [dx, dy]
Then the antinode projected opposite of (x2, y2) is (x2 + dx, y2 + dy) and the antinode opposite of (x1, y1) is (x1 - dx, y1 - dy).

We repeat this process for d13 and d23.




In [17]:
possible_character = set()
for string in data:
    for char in string:
        possible_character.add(char)

print(possible_character)

{'w', 'h', 'C', 'I', 's', 'D', 'j', 'O', 'v', '3', 'u', '4', 'd', 'l', 'r', '.', '7', 'U', 'o', 'P', 'm', 'b', 'k', 'e', 'K', 'W', '0', '6', 'R', 'B', '5', 'L', 'F', '1', 'y', 'Y', '8', 'G', 'S', 't', 'c', '9', 'E', 'i', 'M', 'N', 'V', 'T', 'J', 'f', 'Q', 'Z', 'g', 'A', 'n', 'z', 'a', '2', 'q', 'H', 'p'}


In [19]:
def create_antinodes(p1: Tuple[int], p2: Tuple[int]) -> Tuple[Tuple[int]]:
    """
    Finds the position of a possible antinode

    Args:
        p1: The tuple of coordinates of first point to consider: (x1, y1)
        p2: The tuple of coordinates of second point to consider: (x2, y2)

    Returns:
        The two potential antinode positions
    
    """
    #displacement 
    d = (p2[0] - p1[0], p2[1] - p1[1])
    a1 = (p1[0] - d[0], p1[1] - d[1])
    a2 = (p2[0] + d[0], p2[1] + d[1])

    return a1, a2


def OOB(position: Tuple[int], grid: List[str]) -> bool:
    m = len(grid)
    n = len(grid[0])
    x = position[0]
    y = position[1]

    if x >= m or x < 0:
        return True
    elif y >= n or y < 0 :
        return True
    else:
        return False


def scan_grid(grid: List[str]) -> Dict[str, List[Tuple[int]]]:
    d = defaultdict(list)
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            #The characters in the grid are either "." for empty space or not for an antenna location
            if grid[i][j] != ".":
                d[grid[i][j]].append((i,j))
    return d

In [25]:
@profiler.profile
def part1(grid: List[str]) -> int:
    #Scan grid to find all locations
    antennae = scan_grid(grid)
    antinode_locations = set()

    for locations in antennae.values():
        location_pairs = combinations(locations, 2)
        for pair in location_pairs:
            for antinode in create_antinodes(pair[0], pair[1]):
                if not OOB(antinode, grid):
                    antinode_locations.add(antinode)
            
    return len(antinode_locations)

print(part1(data))

Calling part1: Memory used 262144 kB; Execution Time: 0.000881291925907135 s
357


# Part 2

### Overview 

We must do the same but now antinodes occur anywhere on the line that connects the two antennae that is also within the grid. This includes the antennae themselves.


### Approach

We can use a similar approach but now we need to modify the create_antinodes function to extend the whole line. We will combine it with the out of bounds function. Now the function must find d and reduce it by the greatest common factor between d[0] and d[1], so that we can find the smallest distance to step over. Then we step across this until out of bounds in the positive parameterized direction and then do the same for the negative. 

In [39]:
from math import gcd
def create_antinodes2(p1: Tuple[int], p2: Tuple[int], grid: List[str]) -> Set[Tuple[int]]:
    """
    Finds all positions of a possible antinode

    Args:
        p1: The tuple of coordinates of first point to consider: (x1, y1)
        p2: The tuple of coordinates of second point to consider: (x2, y2)

    Returns:
        The two potential antinode positions
    
    """
    #displacement 
    dx, dy = p2[0] - p1[0], p2[1] - p1[1]
    factor = gcd(dx, dy)
    dx /= factor
    dy /= factor

    antinodes = set()
    
    while not OOB(p1, grid):
        antinodes.add(p1)
        p1 = (p1[0] + dx, p1[1] + dy)

    while not OOB(p2, grid):
        antinodes.add(p2)
        p2 = (p2[0] - dx, p2[1] - dy)
    

    return antinodes


In [43]:
@profiler.profile
def part2(grid: List[str]) -> int:
    antennae = scan_grid(grid)

    result = set()
    for locations in antennae.values():
        location_pairs = combinations(locations, 2)
        for pair in location_pairs:
            result |= create_antinodes2(*pair, grid) #equivalent to result = result.union(create_antinodes2(*pair, grid)) or result = result | create_antinodes2(*pair, grid)
    return len(result)

part2(data)

Calling part2: Memory used 344064 kB; Execution Time: 0.25406112521886826 s


1266